In [5]:
# Задаем дату до которой теоритически будем прогнозировать
end_future_date = '2024-12-31 23:55:00+00:00'

In [2]:
import pandas as pd

# Читаем данные на которых будем обучать модель
df_Pl = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')


In [46]:
# Узнаем последнию известную дату. Нужна для создания промежутка значений будущих дат
end_know_date = df_Pl['time'].iloc[-1]

end_know_date_index = df_Pl[df_Pl['time'] == end_know_date].index[0] + 1


In [40]:
import numpy as np

# Создаем общий датасет с известными и будущими данными нужно для корректного общего нормирования данных

# Добавляем 5 минут чтобы последняя известная дата не попала в список для генерации будущих дат

end_know_date = pd.to_datetime(end_know_date)

# Добавление 5 минут
end_know_date = end_know_date + pd.Timedelta(minutes=5)

# Преобразование обратно в строку
end_know_date = end_know_date.strftime('%Y-%m-%d %H:%M:%S%z')

future_dates = pd.date_range(start=end_know_date, end=end_future_date, freq='5T')

future_df = pd.DataFrame({'time': future_dates, 'P_l': np.nan})

avg_pl = df_Pl['P_l'].mean()

# Заполняем Nan если они есть в датафрейме с известными данными
df_Pl['P_l'].fillna(avg_pl, inplace=True)

# Создаем общий датаферйм с известными и будущими датами для корректной нормировки данных относительно существующих и будущих дат
df_Pl_sacler = pd.concat([df_Pl, future_df], ignore_index=True)


In [41]:
# Функция нормировки дат относительно всех дат(известных и которых хотим прогнощировать)

def normalized_df(df, scaler):
# Преобразование времени в формат datetime
  df['time'] = pd.to_datetime(df['time'])
  # Создание новых столбцов
  df['year'] = df['time'].dt.year
  df['month'] = df['time'].dt.month
  df['day'] = df['time'].dt.day
  df['hour'] = df['time'].dt.hour
  df['minute'] = df['time'].dt.minute

  # Удаление исходной колонки времени
  df = df.drop('time', axis=1)

  # Нормализация данных
  df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
  return df_normalized

# Обратное масштабирование данных времени
def restored_date(df_normalized, scaler):
  df_restored = pd.DataFrame(scaler.inverse_transform(df_normalized), columns=df_normalized.columns)
  df_restored['time'] = pd.to_datetime(df_restored[['year', 'month', 'day', 'hour', 'minute']])
  df_restored = df_restored.drop(['year', 'month', 'day', 'hour', 'minute'], axis=1)
  return df_restored

In [47]:
from sklearn.preprocessing import MinMaxScaler

# Нормализация всех данных существующих и будущих
scaler = MinMaxScaler(feature_range=(0, 1))

df_Pl_normalized_all_dates = normalized_df(df_Pl_sacler, scaler)


In [48]:
# Данные для обучения отнормированные относительно всего промежутка(известных дат и дат для прогноза)

df_Pl_normalized_initial_dates = df_Pl_normalized_all_dates.dropna()


In [49]:
df_Pl_normalized_future_dates = df_Pl_normalized_all_dates.loc[end_know_date_index :]


In [50]:
restored_date(df_Pl_normalized_future_dates, scaler)

,P_l,time
0,NaN,2023-09-11 15:05:00
1,NaN,2023-09-11 15:10:00
2,NaN,2023-09-11 15:15:00
3,NaN,2023-09-11 15:20:00
4,NaN,2023-09-11 15:25:00
...,...,...
137478,NaN,2024-12-31 23:35:00
137479,NaN,2024-12-31 23:40:00
137480,NaN,2024-12-31 23:45:00
137481,NaN,2024-12-31 23:50:00
